In [2]:
!pip install transformers

  Using cached transformers-4.1.1-py3-none-any.whl (1.5 MB)
  Using cached tokenizers-0.9.4-cp38-cp38-macosx_10_11_x86_64.whl (2.0 MB)
Processing /Users/ardsnijders/Library/Caches/pip/wheels/7b/78/f4/27d43a65043e1b75dbddaa421b573eddc67e712be4b1c80677/sacremoses-0.0.43-py3-none-any.whl
     |████████████████████████████████| 284 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 302 kB 3.4 MB/s eta 0:00:01


In [242]:
import torch
import torchtext
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, BertTokenizer
import transformers
import math
import time

torch.set_grad_enabled(False)

class BertSelfAttention_edit(nn.Module):
    def __init__(self, config):
        super().__init__()
        if config.hidden_size % config.num_attention_heads != 0 and not hasattr(config, "embedding_size"):
            raise ValueError(
                "The hidden size (%d) is not a multiple of the number of attention "
                "heads (%d)" % (config.hidden_size, config.num_attention_heads)
            )

        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)

        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)
        self.position_embedding_type = getattr(config, "position_embedding_type", "absolute")
        if self.position_embedding_type == "relative_key" or self.position_embedding_type == "relative_key_query":
            self.max_position_embeddings = config.max_position_embeddings
            self.distance_embedding = nn.Embedding(2 * config.max_position_embeddings - 1, self.attention_head_size)

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(
        self,
        hidden_states,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        output_attentions=False,
    ):
        mixed_query_layer = self.query(hidden_states)

        # If this is instantiated as a cross-attention module, the keys
        # and values come from an encoder; the attention mask needs to be
        # such that the encoder's padding tokens are not attended to.
        if encoder_hidden_states is not None:
            mixed_key_layer = self.key(encoder_hidden_states)
            mixed_value_layer = self.value(encoder_hidden_states)
            attention_mask = encoder_attention_mask
        else:
            mixed_key_layer = self.key(hidden_states)
            mixed_value_layer = self.value(hidden_states)

        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))

        if self.position_embedding_type == "relative_key" or self.position_embedding_type == "relative_key_query":
            seq_length = hidden_states.size()[1]
            position_ids_l = torch.arange(seq_length, dtype=torch.long, device=hidden_states.device).view(-1, 1)
            position_ids_r = torch.arange(seq_length, dtype=torch.long, device=hidden_states.device).view(1, -1)
            distance = position_ids_l - position_ids_r
            positional_embedding = self.distance_embedding(distance + self.max_position_embeddings - 1)
            positional_embedding = positional_embedding.to(dtype=query_layer.dtype)  # fp16 compatibility

            if self.position_embedding_type == "relative_key":
                relative_position_scores = torch.einsum("bhld,lrd->bhlr", query_layer, positional_embedding)
                attention_scores = attention_scores + relative_position_scores
            elif self.position_embedding_type == "relative_key_query":
                relative_position_scores_query = torch.einsum("bhld,lrd->bhlr", query_layer, positional_embedding)
                relative_position_scores_key = torch.einsum("bhrd,lrd->bhlr", key_layer, positional_embedding)
                attention_scores = attention_scores + relative_position_scores_query + relative_position_scores_key

        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        if attention_mask is not None:
            # Apply the attention mask is (precomputed for all layers in BertModel forward() function)
            attention_scores = attention_scores + attention_mask

        # Normalize the attention scores to probabilities.
        attention_probs = nn.Softmax(dim=-1)(attention_scores)
        
        
        
        print(attention_probs.shape)
        
        print(attention_probs[0][0].shape)
        sys.exit()
        
        
        
        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs)

        # Mask heads if we want to
        if head_mask is not None:
            attention_probs = attention_probs * head_mask

        context_layer = torch.matmul(attention_probs, value_layer)

        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)

        outputs = (context_layer, attention_probs) if output_attentions else (context_layer,)
        return outputs

# Store the model we want to use
MODEL_NAME = "bert-base-cased"

# We need to create the model and tokenizer
model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

transformers.models.bert.modeling_bert.BertSelfAttention = BertSelfAttention_edit

print(model._modules['encoder'].layer[0].attention.self)

BertSelfAttention_edit(
  (query): Linear(in_features=768, out_features=768, bias=True)
  (key): Linear(in_features=768, out_features=768, bias=True)
  (value): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


In [250]:
#put the model in eval mode
model.eval()

sent = "The animal did not cross the street because it was too tired"
#Add CLS and SEP Tokens considering it as one sentence
sent = "[CLS] " + sent + " [SEP]"

#apply tokenizer and get ids
tokenized_text = tokenizer.tokenize(sent)
tokenized_text_ids = tokenizer.convert_tokens_to_ids(tokenized_text)

#Convert to torch tensor
input = torch.tensor(tokenized_text_ids)

print("input.shape: ", input.shape) #torch.Size([15]) , N=15 here
input = input.unsqueeze(dim=0) #add a dimension 

print(input)

#Fwd pass - get output
output = model(input) 

input.shape:  torch.Size([14])
tensor([[ 101, 1109, 3724, 1225, 1136, 2771, 1103, 2472, 1272, 1122, 1108, 1315,
         4871,  102]])
torch.Size([1, 12, 14, 14])
torch.Size([14, 14])


SystemExit: 

## Code to create generic Dataset object, code to create dataloaders from Dataset object

In [237]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class OccupationDataset(Dataset):
    """Occupation Classification dataset."""

    def __init__(self, dataset, anonymization=False, transform=None):
        """
        Args:
            
        """
        self.path = './src/classification_tasks/data/occupation-classification/' + dataset + '.txt'     
        self.text = pd.read_csv(self.path, sep="\t", header=None)
        self.transform = transform
        self.anonymization = anonymization
        self.impermissible = ["he", "she", "her", "his", "him"]

    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        label, sentence = self.text.iloc[idx]
        
        if self.anonymization:
            
            splitted_sentence = sentence.split()
            sentence = ' '.join([i for i in splitted_sentence if i not in self.impermissible])

        sample = {'sentence' : sentence, 'label' : label}
        return sample

occupation_train = OccupationDataset(dataset='train', anonymization=True)
occupation_train_loader = torch.utils.data.DataLoader(occupation_train, batch_size=2, shuffle=False, num_workers=0)

In [240]:
for i, batch in enumerate(occupation_train_loader):
    
    labels = batch['label']
    sents = batch['sentence']
    
    print(sents)
    
    
    break

['has subspecialty interests in arthroscopic/keyhole and sports surgery, knee and hip replacements, cartilage regeneration and stem cell therapeutics in orthopaedics.', 'ms. staples practices medicine in spring lake park, mn and specializes in orthopaedic surgery. ms. staples is affiliated with unity hospital, north memorial medical center and mercy hospital.']


# pseudocode of possible train loop:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


def tokenize_batch(sents):
    
    tokenized_text = tokenizer.tokenize(sents)
    tokenized_text_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
    
    return tokenized_text, tokenized_te


In [176]:
epochs = 1

for epoch in range(epochs):
    for i, batch in enumerate(occupation_train_loader):
        
        labels = batch['label']
        sents = batch['sentence']
        
        #TODO: Tokenize sentences, convert to ids to sequences of ids
        #TODO: Generate self-attention mask for each sequence
        
        tokenized_sents = tokenize_batch(sents)
        
        
        
#         print(sentence)
        
#         print(tokenized_text)
        
        for ids in tokenized_text_ids:
            print(tokenizer.decode(ids))
        
        break
        
        
        

she
graduated
from
the
university
of
ca
##li
##fo
##rn
##ia
,
da
##vis
in
2004
and
is
certified
by
the
national
commission
on
certification
of
physician
assistants
.
p
##ors
##cha
moved
to
the
ta
##hoe
/
truck
##ee
area
in
1996
and
joined
ta
##hoe
forest
health
system
in
2011
.
m
##s
.
sick
##els
practices
medicine
in
color
##ado
springs
,
co
and
specializes
in
internal
medicine
.
m
##s
.
sick
##els
is
affiliated
with
pen
##rose
saint
f
##ran
##cis
health
services
.
m
##s
.
sick
##els
speaks
en
##gli
##sh
and
span
##ish
.


In [249]:
sent = "he she her his him"
tokenized_text = tokenizer.tokenize(sent)
tokenized_text_ids = tokenizer.convert_tokens_to_ids(tokenized_text)

tokenized_text, tokenized_text_ids

(['he', 'she', 'her', 'his', 'him'], [1119, 1131, 1123, 1117, 1140])

In [ ]:
impermissible = ""